# Đánh giá mô tả mã HS bằng LLM as a Judge + Checklist
Notebook này thực hiện các bước:
1. Đọc dữ liệu mô tả mã HS từ file CSV.
2. Nhóm dữ liệu theo prefix mã HS.
3. Xây dựng prompt theo phương pháp LLM as a Judge + Checklist.
4. Gọi LLM để đánh giá từng nhóm.
5. Lưu kết quả ra file CSV.
6. Hỗ trợ debug từng bước.

In [1]:
# Import các thư viện cần thiết
import os
import json
import pandas as pd
from collections import defaultdict
from dotenv import load_dotenv
from typing import List, Dict

In [2]:
# Đọc dữ liệu từ file CSV
data_path = "../../data/description data/mota_motamoi.csv"
df = pd.read_csv(data_path, dtype=str)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
df = df[['mahs', 'mo_ta', 'mo_ta_moi']]
df.head()

,mahs,mo_ta,mo_ta_moi
0,01012900,"Ngựa, lừa, la sống/- Ngựa:/- - Loại khác","Mã 01012900 thuộc nhóm 0101 (Ngựa, lừa, la sốn..."
1,01013010,"Ngựa, lừa, la sống/- Lừa:/- - Loại thuần chủng...","Mã 01013010 thuộc nhóm 0101 (Ngựa, lừa, la sốn..."
2,01013090,"Ngựa, lừa, la sống/- Lừa:/- - Loại khác","Mã 01013090 thuộc nhóm 0101 (Ngựa, lừa, la sốn..."
3,01019000,"Ngựa, lừa, la sống/- Loại khác","Mã 01019000 thuộc nhóm 0101 (Ngựa, lừa, la sốn..."
4,01022100,Động vật sống họ trâu bò/- Gia súc:/- - Loại t...,"Mã 01022100 thuộc nhóm 0102, mô tả động vật số..."


In [3]:
# Nhóm dữ liệu theo prefix 4 ký tự đầu của mã HS
def group_by_hs_prefix(df):
    grouped = defaultdict(list)
    for _, row in df.iterrows():
        prefix = str(row['mahs'])[:4]
        grouped[prefix].append(row.to_dict())
    return grouped

grouped_data = group_by_hs_prefix(df)
print(f"Số nhóm prefix: {len(grouped_data)}")
# Hiển thị ví dụ 1 nhóm
for prefix, items in grouped_data.items():
    print(f"Prefix: {prefix}, số lượng: {len(items)}")
    print(items[0])
    break

Số nhóm prefix: 4
Prefix: 0101, số lượng: 4
{'mahs': '01012900', 'mo_ta': 'Ngựa, lừa, la sống/- Ngựa:/- - Loại khác', 'mo_ta_moi': 'Mã 01012900 thuộc nhóm 0101 (Ngựa, lừa, la sống), mô tả các loại ngựa khác, không bao gồm các loại được liệt kê cụ thể ở các mã khác.'}


In [ ]:
# Xây dựng prompt cho 1 nhóm prefix (ví dụ nhóm đầu tiên)
def build_prompt_for_evaluation(prefix: str, items: List[Dict]) -> str:
    item_list = "".join(
        f"- {item['mahs']}:\n"
        f"  + Mô tả gốc: {item['mo_ta']}\n"
        f"  + Mô tả mới: {item['mo_ta_moi']}\n"
        for item in items
    )
    prompt = f'''Bạn là chuyên gia đánh giá chất lượng mô tả hàng hóa theo mã HS code.
      Dưới đây là các cặp mô tả gốc và mô tả mới thuộc nhóm {prefix}:
      \n\n{item_list}\n\n---\n\n##
        Nhiệm vụ:\n\nĐánh giá **từng cặp mô tả** (gốc và mới) 
        dựa theo các tiêu chí sau (chấm từng tiêu chí Đúng/Sai):
        \n\n### Checklist đánh giá:\n1. **Giữ đúng ngữ nghĩa** từ mô tả gốc
          (`dung_noi_dung`)\n2. **Không suy diễn / bổ sung ngoài** nội dung ban đầu (`khong_them_thong_tin`)\n3. **Ngôn ngữ đời thường** (tự nhiên, dễ hiểu) (`ngon_ngu_tu_nhien`)\n4. **Tránh thuật ngữ \"level\", \"tầng\", \"phân cấp\"...** (`tranh_thuat_ngu`)\n5. **Phân biệt rõ giữa các mã trong nhóm** (`phan_biet`)\n6. **Không xuất hiện từ ngoài mo_ta gốc** (ngoại trừ ngôn ngữ phụ trợ) (`khong_tu_ngoai`)\n7. **Định dạng hợp lý, rõ ràng** (`dinh_dang`)\n\n---\n\n## Đầu ra mong muốn:\n\nVới mỗi mã HS, trả về:\n- `mahs`\n- `dung_noi_dung`: true/false\n- `khong_them_thong_tin`: true/false\n- `ngon_ngu_tu_nhien`: true/false\n- `tranh_thuat_ngu`: true/false\n- `phan_biet`: true/false\n- `khong_tu_ngoai`: true/false\n- `dinh_dang`: true/false\n- `danh_gia`: "Tốt", "Chấp nhận được", hoặc "Không đạt"\n- `giai_thich`: Lý do ngắn gọn\n\n---\n\n## Định dạng JSON:\n\n{{\n  "{prefix}": [\n    {{\n      "mahs": "...",\n      "dung_noi_dung": true,\n      "khong_them_thong_tin": true,\n      "ngon_ngu_tu_nhien": true,\n      "tranh_thuat_ngu": true,\n      "phan_biet": true,\n      "khong_tu_ngoai": true,\n      "dinh_dang": true,\n      "danh_gia": "...",\n      "giai_thich": "..."\n    }}, ...\n  ]\n}}\n\n---\n\n**Lưu ý:** Chỉ đánh giá dựa trên checklist, không áp đặt phong cách cá nhân.\n'''
    return prompt

# Lấy 1 nhóm ví dụ
prefix, items = next(iter(grouped_data.items()))
prompt = build_prompt_for_evaluation(prefix, items)
print(prompt)

Bạn là chuyên gia đánh giá chất lượng mô tả hàng hóa theo mã HS code. Dưới đây là các cặp mô tả gốc và mô tả mới thuộc nhóm 0101:

- 01012900:
  + Mô tả gốc: Ngựa, lừa, la sống/- Ngựa:/- - Loại khác
  + Mô tả mới: Mã 01012900 thuộc nhóm 0101 (Ngựa, lừa, la sống), mô tả các loại ngựa khác, không bao gồm các loại được liệt kê cụ thể ở các mã khác.
- 01013010:
  + Mô tả gốc: Ngựa, lừa, la sống/- Lừa:/- - Loại thuần chủng để nhân giống
  + Mô tả mới: Mã 01013010 thuộc nhóm 0101 (Ngựa, lừa, la sống), mô tả lừa thuần chủng được sử dụng cho mục đích nhân giống.
- 01013090:
  + Mô tả gốc: Ngựa, lừa, la sống/- Lừa:/- - Loại khác
  + Mô tả mới: Mã 01013090 thuộc nhóm 0101 (Ngựa, lừa, la sống), mô tả các loại lừa khác, không phải loại thuần chủng dùng để nhân giống.
- 01019000:
  + Mô tả gốc: Ngựa, lừa, la sống/- Loại khác
  + Mô tả mới: Mã 01019000 thuộc nhóm 0101 (Ngựa, lừa, la sống), mô tả các loại động vật thuộc họ này khác, không phải ngựa hay lừa.


---

## Nhiệm vụ:

Đánh giá **từng cặp mô

In [5]:
# (Tuỳ chọn) Chuẩn bị hàm gọi LLM Gemini (chỉ chạy được nếu đã cấu hình API key và cài đúng thư viện)
from google import genai
def call_llm(prompt, model="gemini-2.0-flash-001"):
    load_dotenv()
    api_key = os.getenv("GOOGLE_GENAI_API_KEY")
    if not api_key:
        raise ValueError("API key không được tìm thấy. Vui lòng đặt GOOGLE_GENAI_API_KEY trong .env.")
    client = genai.Client(api_key=api_key)
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config={
            "response_mime_type": "application/json"
        },
    )
    return response.text

In [9]:
# (Tuỳ chọn) Gọi LLM cho 1 nhóm và hiển thị kết quả thô
result = call_llm(prompt)
print(result)
print("(Chạy cell này nếu đã cấu hình API key và cài đúng thư viện google-genai)")

{
  "0101": [
    {
      "mahs": "01012900",
      "dung_noi_dung": true,
      "khong_them_thong_tin": true,
      "ngon_ngu_tu_nhien": true,
      "tranh_thuat_ngu": true,
      "phan_biet": true,
      "khong_tu_ngoai": true,
      "dinh_dang": true,
      "danh_gia": "Tốt",
      "giai_thich": "Mô tả mới giữ đúng nội dung, không thêm thông tin, ngôn ngữ tự nhiên và rõ ràng."
    },
    {
      "mahs": "01013010",
      "dung_noi_dung": true,
      "khong_them_thong_tin": true,
      "ngon_ngu_tu_nhien": true,
      "tranh_thuat_ngu": true,
      "phan_biet": true,
      "khong_tu_ngoai": true,
      "dinh_dang": true,
      "danh_gia": "Tốt",
      "giai_thich": "Mô tả mới giữ đúng nội dung, không thêm thông tin, ngôn ngữ tự nhiên và rõ ràng."
    },
    {
      "mahs": "01013090",
      "dung_noi_dung": true,
      "khong_them_thong_tin": true,
      "ngon_ngu_tu_nhien": true,
      "tranh_thuat_ngu": true,
      "phan_biet": true,
      "khong_tu_ngoai": true,
      "dinh_dang":

In [7]:
# (Tuỳ chọn) Parse kết quả JSON từ LLM và hiển thị dạng bảng
# raw_result = result  # Gán kết quả thô từ LLM vào biến này
# try:
#     data = json.loads(raw_result)
#     if isinstance(data, dict):
#         for v in data.values():
#             if isinstance(v, list):
#                 display(pd.DataFrame(v))
#     elif isinstance(data, list):
#         display(pd.DataFrame(data))
#     else:
#         print("Kết quả không đúng định dạng mong muốn")
# except Exception as e:
#     print("Lỗi parse JSON:", e)